# Lab 4: Azure AI Foundry Agent Service - Workflow Pattern

## Overview

This notebook demonstrates deploying a Multi-Agent system based on **Workflow Pattern** using **Azure AI Foundry Agent Service**.

### 🔑 Core Implementation Approach

- **Azure AI Foundry Agent Service**: Same Agent foundation as Lab 3
- **Workflow Pattern**: Orchestration using Router + Executor pattern
- **AI-based Routing**: Intent classification using LLM
- **Workflow Context**: Message-based state management

### 💡 Differences from Lab 3

| Feature | Lab 3 (Connected Agent) | Lab 4 (This Notebook) |
|---------|------------------------|----------------------|
| **Agent Foundation** | ✅ Foundry Agent Service | ✅ Foundry Agent Service |
| **Workflow** | Connected Agent (Handoff) | **Workflow Pattern (Router+Executor)** |
| **Routing Method** | `handoff_to_agent()` API | **Router Executor Function** |
| **Execution Flow** | Main → Handoff → Sub Agent | **Router → Executor → Output** |
| **State Management** | Thread-based | **Workflow Context-based** |
| **Parallel Execution** | Sequential Handoff | **Orchestrator Parallel Capable** |

> **✅ Common Ground**: Both labs use the **same Azure AI Foundry Agent Service**.
> 
> **🎯 Difference**: Different **Agent orchestration patterns** (Connected Agent vs Workflow Pattern).
> 
> **⚡ Advantage**: Workflow Pattern enables advanced orchestration including complex conditional branching, parallel execution, and loops.

### Architecture
```
┌────────────────────────────────────────────────────────────┐
│        Agent Framework - Workflow Pattern                 │
│                                                            │
│  ┌──────────────────────────────────────────┐             │
│  │        Router Executor                   │             │
│  │   (AI-based Intent Classification)       │             │
│  └────┬──────┬────────┬────────────┬────────┘             │
│       │      │        │            │                      │
│   ┌───▼──┐ ┌▼───┐  ┌─▼────┐   ┌──▼────────┐             │
│   │ Tool │ │Research│ │General│ │Orchestrator│            │
│   │Exec  │ │Executor│ │Executor│ │Executor  │             │
│   └───┬──┘ └┬───┘  └─┬────┘   └──┬────────┘             │
│       │     │        │            │                      │
│   ┌───▼─────▼────────▼────────────▼──────┐               │
│   │      Workflow Context                │               │
│   │   (Message Passing & Output)         │               │
│   └──────────────────────────────────────┘               │
│                                                            │
│   External Resources:                                     │
│   ┌──────────────┐    ┌────────────────┐                 │
│   │  MCP Server  │    │  Azure AI      │                 │
│   │  (Tools)     │    │  Search (RAG)  │                 │
│   └──────────────┘    └────────────────┘                 │
└────────────────────────────────────────────────────────────┘
```

### Key Differences: Foundry Agent vs Agent Framework

| Feature | Foundry Agent (Lab 3) | Agent Framework (Lab 4) |
|---------|----------------------|-------------------------|
| **Pattern** | Connected Agent (Handoff) | Workflow Executor Pattern |
| **Message Flow** | Thread-based conversation | Workflow Context streaming |
| **Inter-Agent Communication** | Agent API call (handoff) | Workflow message routing |
| **Execution Model** | Synchronous handoff | Async executor graph |
| **Scalability** | Handoff setup when adding agents | Add Executor & Edge |
| **Tracing** | Agent Service built-in tracing | Custom OpenTelemetry |
| **State Management** | Thread state persistence | Workflow Context-based |
| **Complexity** | Medium (Azure managed) | Low (code-centric) |

### Advantages of Workflow Pattern

1. ✅ **Flexible Message Routing**: Control complex flows with simple conditionals
2. ✅ **Asynchronous Execution**: Support for parallel execution (Orchestrator)
3. ✅ **Clear Execution Graph**: Visual structure definition with WorkflowBuilder
4. ✅ **Easy Local Development**: Minimal Azure dependencies, fast testing
5. ✅ **Custom Control**: Fine-grained error handling and logging

### Python Module Structure

```
src/agent_framework/
├── main_agent_workflow.py  - Workflow definition and Executor implementation
├── tool_agent.py           - Tool Agent class (MCP)
├── research_agent.py       - Research Agent class (RAG)
├── api_server.py           - FastAPI HTTP server
├── masking.py              - Masking utilities
└── requirements.txt        - Python dependencies
```

### Learning Objectives

1. ✅ Understand Microsoft Agent Framework Workflow Pattern
2. ✅ Implement Executor-based Multi-Agent system
3. ✅ Message routing through Workflow Context
4. ✅ Parallel execution (Orchestrator) pattern
5. ✅ Deploy Agent Framework-based system to ACA
6. ✅ Compare Foundry Agent and Agent Framework


---

## ⚙️ Before You Start

**Select Python Kernel:**

1. Click **"Select Kernel"** at the top right of the notebook
2. Select **"Python Environments..."**
3. Select **`.venv (Python 3.x.x)`** (virtual environment created at project root)

> 💡 **GitHub Codespaces**: The `.venv` environment is automatically created in Codespaces.  
> If you don't see `.venv`, create it in the terminal with `python -m venv .venv`.

---


## 1. Environment Setup & Authentication

### Tenant ID Configuration Guide

**In most cases**: You don't need to specify a tenant ID. Leave the `tenant_id` variable as `"<YOUR_TENANT_ID>"` or `None` and proceed.

**When tenant ID is required**:
- ✅ When you have access to multiple organization (company) Azure tenants
- ✅ When you need to work with resources from a specific organization only
- ✅ When you get "multiple tenants" related errors during login

**How to find your Tenant ID**:
- Azure Portal → Azure Active Directory → Overview → Copy Tenant ID
- Or contact your organization administrator


In [ ]:
import sys, subprocess, os, json
import platform

# Set PATH according to operating system
system = platform.system()
if system == 'Darwin':  # macOS
    extra_paths = '/opt/homebrew/bin:/usr/local/bin'
elif system == 'Linux':  # Linux / Codespaces
    extra_paths = '/usr/local/bin:/usr/bin:/home/codespace/.local/bin'
else:  # Windows
    extra_paths = ''

if extra_paths:
    os.environ['PATH'] = extra_paths + ':' + os.environ.get('PATH', '')

def check(cmd, name):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, timeout=3, env=os.environ)
        print(f"{'✓' if result.returncode == 0 else '✗'} {name}")
    except Exception as e:
        print(f"✗ {name}")

print("=== Prerequisites Check ===")
print(f"✓ Python {sys.version.split()[0]} ({system})")
check("az --version", "Azure CLI")
check("docker --version", "Docker")
print("="*50)


In [ ]:
import subprocess, json

print("=== Azure Authentication ===")
print("ℹ️  Checking authentication status and logging in if needed.\n")

# Enter your tenant ID here (optional)
# Example: tenant_id = "16b3c013-d300-468d-ac64-7eda0820b6d3"
tenant_id = "<YOUR_TENANT_ID>"  # Or set to None to use default tenant

# Check Azure CLI authentication status
az_account = subprocess.run("az account show", shell=True, capture_output=True, text=True)

if az_account.returncode == 0:
    account_info = json.loads(az_account.stdout)
    print(f"✅ Azure CLI authentication successful (using existing session)")
    print(f"   Subscription: {account_info.get('name', 'N/A')}")
    print(f"   Tenant: {account_info.get('tenantId', 'N/A')}")
else:
    print("⚠️  Azure CLI authentication required. Opening browser...")
    # If tenant ID is configured, log in to that tenant
    if tenant_id and tenant_id != "<YOUR_TENANT_ID>":
        az_login = subprocess.run(f"az login --tenant {tenant_id}", shell=True)
    else:
        az_login = subprocess.run("az login", shell=True)
    
    if az_login.returncode == 0:
        print("✅ Azure CLI login successful")
    else:
        raise Exception("❌ Azure CLI login failed")

print("="*50)


In [ ]:
# Load configuration file
config_path = "config.json"
with open(config_path) as f:
    config = json.load(f)

# Set environment variables
RESOURCE_GROUP = config["resource_group"]
LOCATION = config["location"]
PROJECT_CONNECTION_STRING = config["project_connection_string"]
SEARCH_ENDPOINT = config["search_endpoint"]
SEARCH_INDEX = config["search_index"]
CONTAINER_REGISTRY = config["container_registry_endpoint"]
CONTAINER_ENV_ID = config["container_apps_environment_id"]
MCP_ENDPOINT = config.get("mcp_endpoint", "")

# Convert PROJECT_CONNECTION_STRING to simple format
simple_project_conn = PROJECT_CONNECTION_STRING.split(';')[0] if PROJECT_CONNECTION_STRING else ""

print("=== Configuration Loaded ===")
print(f"Resource Group: {RESOURCE_GROUP}")
print(f"Location: {LOCATION}")
print(f"Search Index: {SEARCH_INDEX}")
print(f"MCP Endpoint: {MCP_ENDPOINT if MCP_ENDPOINT else 'Not configured'}")
print(f"Container Registry: {CONTAINER_REGISTRY}")
print("="*50)


## 2. Install Required Packages

Installs Azure AI related required packages. Most packages may already be installed if running in GitHub Codespace.


In [ ]:
# Install required packages
import subprocess
import sys

packages = [
    "azure-ai-projects",
    "azure-ai-inference",
    "azure-search-documents",
    "azure-identity",
    "openai",
    "python-dotenv",
    "requests",
    "fastapi",
    "uvicorn",
    "httpx"
]

print("=== Installing Required Packages ===\n")

for package in packages:
    print(f"Installing {package}...")
    result = subprocess.run(
        [sys.executable, "-m", "pip", "install", "-q", package],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        print(f"✅ {package} installed")
    else:
        print(f"⚠️  {package} may already be installed or failed to install")

print("\n" + "="*50)
print("✅ Package installation completed!")


## 3. Get Azure AI Search Key

Retrieves the Azure AI Search admin key to be used by the Research Agent.


In [ ]:
# Get AI Search admin key
search_name = config["search_service_name"]

search_key_cmd = f"""
az search admin-key show \
    --resource-group {RESOURCE_GROUP} \
    --service-name {search_name} \
    --query primaryKey -o tsv
"""

result = subprocess.run(search_key_cmd, shell=True, capture_output=True, text=True)
SEARCH_KEY = result.stdout.strip()

if SEARCH_KEY:
    print(f"✅ Search key retrieved: {SEARCH_KEY[:10]}...")
    os.environ['SEARCH_KEY'] = SEARCH_KEY
else:
    print("❌ Failed to retrieve search key")


## 4. Build Agent Framework Container

### Container Configuration

- **Framework**: Microsoft Agent Framework (Workflow Pattern)
- **Port**: 8000
- **Endpoints**:
  - `/health` - Health check
  - `/` - Agent Framework status information
  - `/chat` - Workflow execution endpoint (POST)

### Workflow Executors

- **Router Executor**: AI-based intent classification and routing
- **Tool Executor**: MCP tool execution
- **Research Executor**: RAG-based knowledge retrieval
- **General Executor**: General conversation
- **Orchestrator Executor**: Parallel execution and result aggregation

### MCP Server Features

- **Real-time Weather Information**:
  - `get_weather(location)` - Accurate real-time weather for cities worldwide
  - Data source: wttr.in API (free, highly reliable)
  - Supported languages: Both Korean/English city names (e.g., 'Seoul', '서울')
  - Provided info: Temperature, feels-like temp, weather condition, humidity, wind speed/direction, observation time

### Environment Variable Configuration

The following variables are automatically configured in the `.env` file:

- `AZURE_AI_PROJECT_ENDPOINT` - Azure AI Project endpoint
- `AZURE_AI_MODEL_DEPLOYMENT_NAME` - Model deployment name (gpt-4o)
- `SEARCH_ENDPOINT`, `SEARCH_INDEX` - Azure AI Search configuration
- `MCP_ENDPOINT` - MCP Server endpoint
- `APPLICATIONINSIGHTS_CONNECTION_STRING` - Application Insights (Analytics)
- `OTEL_*` - OpenTelemetry configuration (Tracing)
- `AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED` - Prompt/Completion recording


In [ ]:
# Container Registry login
registry_name = CONTAINER_REGISTRY.split('.')[0]

print("=== Container Registry Login ===")
login_cmd = f"az acr login --name {registry_name}"
result = subprocess.run(login_cmd, shell=True, capture_output=True, text=True)

if result.returncode == 0:
    print(f"✅ Logged in to {registry_name}")
else:
    print(f"❌ Login failed: {result.stderr}")
print("="*50)


In [ ]:
# Create .env file (for Agent Framework Container)
print("=== Creating .env file for Agent Framework Container ===\n")

# 1. Get Application Insights Connection String
print("📊 Getting Application Insights connection string...")
appinsights_cmd = f"""
az monitor app-insights component show \
    --resource-group {RESOURCE_GROUP} \
    --query "[0].connectionString" -o tsv
"""

result = subprocess.run(appinsights_cmd, shell=True, capture_output=True, text=True)

if result.returncode == 0 and result.stdout.strip():
    APP_INSIGHTS_CONN_STR = result.stdout.strip()
    print(f"✅ Application Insights connection string retrieved\n")
else:
    print(f"⚠️  Could not get Application Insights connection string")
    print(f"   Proceeding without Application Insights\n")
    APP_INSIGHTS_CONN_STR = ""

# 2. Get model configuration (from config.json)
model_deployment_name = config.get("model_deployment_name", "gpt-4o")
model_version = config.get("model_version", "2024-11-20")
model_capacity = config.get("model_capacity", 50)
print(f"📦 Model Configuration:")
print(f"   Deployment Name: {model_deployment_name}")
print(f"   Model Version: {model_version}")
print(f"   Capacity (TPM): {model_capacity}")
print(f"   (from config.json - set in Lab 1 infrastructure deployment)\n")

# 3. Create .env file
env_content = f"""# Azure AI Project Configuration (Microsoft Agent Framework)
AZURE_AI_PROJECT_ENDPOINT={simple_project_conn}

# Model Configuration
# The model deployment name and version from Azure OpenAI
# These values are automatically set from Lab 1 infrastructure deployment (infra/main.bicep)
AZURE_AI_MODEL_DEPLOYMENT_NAME={model_deployment_name}
AZURE_AI_MODEL_VERSION={model_version}

# MCP Server Configuration
MCP_ENDPOINT={MCP_ENDPOINT if MCP_ENDPOINT else ''}

# Azure AI Search Configuration (for Research Agent with RAG)
SEARCH_ENDPOINT={SEARCH_ENDPOINT}
SEARCH_INDEX={SEARCH_INDEX}
SEARCH_KEY={SEARCH_KEY}

# Application Insights Configuration (for Application Analytics)
APPLICATIONINSIGHTS_CONNECTION_STRING={APP_INSIGHTS_CONN_STR}

# OpenTelemetry Configuration (for Tracing)
OTEL_SERVICE_NAME=agent-framework-workflow
OTEL_TRACES_EXPORTER=azure_monitor
OTEL_METRICS_EXPORTER=azure_monitor
OTEL_LOGS_EXPORTER=azure_monitor
OTEL_PYTHON_LOGGING_AUTO_INSTRUMENTATION_ENABLED=true
AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED=true

# Masking / PII Handling
AGENT_MASKING_MODE=standard  # off|standard|strict
"""

env_file_path = "src/agent_framework/.env"

try:
    with open(env_file_path, 'w') as f:
        f.write(env_content)
    
    print(f"✅ Created {env_file_path}")
    print("\n📋 Environment variables:")
    for line in env_content.strip().split('\n'):
        if line and not line.startswith('#'):
            key = line.split('=')[0]
            print(f"   • {key}")
    
    print("\n💡 This file will be included in the Docker image.")
    
    if APP_INSIGHTS_CONN_STR:
        print("\n✅ Application Insights configured!")
        print("   → OpenTelemetry Tracing enabled")
        print("   → GenAI content recording enabled")
    else:
        print("\n⚠️  Application Insights not configured")
        print("   → Analytics won't work but Agent will function normally.")
    
    print(f"\n🔍 Azure AI Search configured!")
    print(f"   → RAG (Retrieval-Augmented Generation) enabled")
    print(f"   → Research Agent can search knowledge base")
    print(f"\n🤖 Model: {model_deployment_name} (version {model_version}, capacity {model_capacity}K TPM)")
    
except Exception as e:
    print(f"❌ Failed to create .env file: {e}")

print("\n" + "="*60)


In [ ]:
# Build and push Agent Framework Container image
import time

framework_image = f"{CONTAINER_REGISTRY}/agent-framework:latest"

print("=== Building Agent Framework Image ===")
print(f"Image: {framework_image}\n")

# Build (linux/amd64 platform for Azure Container Apps)
build_cmd = f"docker build --platform linux/amd64 -t {framework_image} ./src/agent_framework"
print("🔨 Building image (linux/amd64)...")
start_time = time.time()

result = subprocess.run(build_cmd, shell=True, capture_output=True, text=True)
elapsed = time.time() - start_time

if result.returncode == 0:
    print(f"✅ Build successful ({elapsed:.1f}s)")
    print(f"   Framework: Microsoft Agent Framework (Workflow Pattern)")
else:
    print(f"❌ Build failed: {result.stderr}")
    
# Push
if result.returncode == 0:
    print("\n📤 Pushing image to registry...")
    push_cmd = f"docker push {framework_image}"
    result = subprocess.run(push_cmd, shell=True, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"✅ Push successful")
    else:
        print(f"❌ Push failed: {result.stderr}")

print("="*50)


## 5. Verify Azure Resources

Verify required Azure resources before deploying Agent Framework Service.

**Verification Items:**
- ✅ Azure AI Project resource ID


In [ ]:
# Verify Azure AI Project resource ID
print("=== Verifying Azure Resources ===\n")

# Extract project_name from URL
if '/api/projects/' in simple_project_conn:
    project_name = simple_project_conn.split('/api/projects/')[-1].strip()
else:
    project_name = None

print(f"📋 Project Information:")
print(f"   Resource Group: {RESOURCE_GROUP}")
print(f"   Project Name: {project_name if project_name else 'Not found'}\n")

# Get AI Project resource ID
print("🔍 Finding AI Project resource...")
if project_name:
    ai_project_cmd = f"""
    az resource list \
        --resource-group {RESOURCE_GROUP} \
        --query "[?contains(name, '{project_name}') && type=='Microsoft.CognitiveServices/accounts/projects'].id" -o tsv
    """
else:
    ai_project_cmd = f"""
    az resource list \
        --resource-group {RESOURCE_GROUP} \
        --query "[?type=='Microsoft.CognitiveServices/accounts/projects'].id | [0]" -o tsv
    """

result = subprocess.run(ai_project_cmd, shell=True, capture_output=True, text=True)
if result.returncode == 0 and result.stdout.strip():
    ai_project_resource_id = result.stdout.strip()
    print(f"   ✅ AI Project Resource ID:")
    print(f"   {ai_project_resource_id}\n")
else:
    print(f"   ❌ Could not find AI Project")
    raise Exception("AI Project not found")

print("✅ All required resources verified!")
print("="*60)


## 6. Deploy Agent Framework Service

Deploy Agent Framework Service and **automatically** configure Managed Identity immediately after deployment.

### Automatic Actions Performed

1. ✅ Deploy Container App (starting with replicas=0)
2. ✅ Enable System-assigned Managed Identity
3. ✅ Assign Azure AI User role (AI Project scope)
4. ✅ Permission propagation wait notice

> 💡 **Important**: Deployment and permission setup are handled together, taking approximately 3-4 minutes to complete.


In [ ]:
# Deploy Agent Framework Service to Container App + Configure Managed Identity permissions
framework_app_name = "agent-framework"

print("=== Deploying Agent Framework Service to ACA ===")
print(f"App Name: {framework_app_name}\n")

print("💡 Environment variables are already included in the Docker image.\n")

# 1. Deploy Container App (with Managed Identity, replicas=0)
deploy_cmd = f"""
az containerapp create \
    --name {framework_app_name} \
    --resource-group {RESOURCE_GROUP} \
    --environment {CONTAINER_ENV_ID.split('/')[-1]} \
    --image {framework_image} \
    --target-port 8000 \
    --ingress external \
    --min-replicas 0 \
    --max-replicas 3 \
    --cpu 1.0 \
    --memory 2.0Gi \
    --registry-server {CONTAINER_REGISTRY} \
    --system-assigned \
"""

print("🚀 Deploying Agent Framework Service with Managed Identity...")
print("   (Starting with 0 replicas to configure permissions first)")
result = subprocess.run(deploy_cmd, shell=True, capture_output=True, text=True, timeout=180)

if result.returncode == 0:
    print("✅ Deployment successful\n")
    
    # Get endpoint
    show_cmd = f"""
    az containerapp show \
        --name {framework_app_name} \
        --resource-group {RESOURCE_GROUP} \
        --query properties.configuration.ingress.fqdn -o tsv
    """
    result = subprocess.run(show_cmd, shell=True, capture_output=True, text=True)
    FRAMEWORK_ENDPOINT = f"https://{result.stdout.strip()}"
    
    print(f"🌐 Agent Framework Endpoint: {FRAMEWORK_ENDPOINT}")
    
    # Update config
    config['agent_framework_endpoint'] = FRAMEWORK_ENDPOINT
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)
    print("✅ Config updated\n")
    
    # 2. Get Managed Identity Principal ID
    print("="*60)
    print("🔐 Configuring Permissions\n")
    
    print("1️⃣ Getting Managed Identity Principal ID...")
    identity_cmd = f"""
    az containerapp show \
        --name {framework_app_name} \
        --resource-group {RESOURCE_GROUP} \
        --query identity.principalId -o tsv
    """
    
    result = subprocess.run(identity_cmd, shell=True, capture_output=True, text=True)
    if result.returncode == 0 and result.stdout.strip():
        principal_id = result.stdout.strip()
        print(f"   ✅ Principal ID: {principal_id}\n")
    else:
        print(f"   ❌ Failed to get Principal ID\n")
        raise Exception("Failed to get Managed Identity Principal ID")
    
    # 3. Assign Azure AI User role
    print("2️⃣ Assigning 'Azure AI User' role to AI Project...")
    print(f"   Scope: {ai_project_resource_id}")
    role_assignment_cmd = f"""
    az role assignment create \
        --assignee {principal_id} \
        --role "Azure AI User" \
        --scope {ai_project_resource_id}
    """
    
    result = subprocess.run(role_assignment_cmd, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print("   ✅ Azure AI User role assigned\n")
    elif "already exists" in result.stderr.lower():
        print("   ✅ Azure AI User role already exists\n")
    else:
        print(f"   ❌ Role assignment FAILED!")
        print(f"   Error: {result.stderr}\n")
    
    # 4. Verify permissions
    print("3️⃣ Verifying role assignments...\n")
    import time
    time.sleep(5)
    
    role_check_cmd = f"""
    az role assignment list \
        --assignee {principal_id} \
        --query "[].{{role:roleDefinitionName, scope:scope}}" -o json
    """
    
    result = subprocess.run(role_check_cmd, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        import json as json_lib
        current_roles = json_lib.loads(result.stdout)
        
        print(f"   📋 Current Role Assignments ({len(current_roles)} total):\n")
        
        required_roles = {"Azure AI User (AI Project)": False}
        
        for role in current_roles:
            scope_parts = role['scope'].split('/')
            resource_name = scope_parts[-1] if scope_parts else 'Unknown'
            role_name = role['role']
            
            print(f"      • {role_name} → {resource_name}")
            
            if role_name == "Azure AI User":
                if "projects" in role['scope'] or ai_project_resource_id in role['scope']:
                    required_roles["Azure AI User (AI Project)"] = True
        
        print(f"\n   🔍 Required Roles Verification:")
        all_roles_ok = True
        for role_name, assigned in required_roles.items():
            status = "✅" if assigned else "❌"
            print(f"      {status} {role_name}")
            if not assigned:
                all_roles_ok = False
        
        if all_roles_ok:
            print(f"\n   ✅ All required roles are assigned!\n")
        else:
            print(f"\n   ❌ Some required roles are missing!\n")
    
    # 5. Permission propagation wait notice
    print("="*60)
    print("4️⃣ Permissions assigned - waiting for propagation...\n")
    print("⚠️  Azure RBAC permission propagation can take up to 5-10 minutes.")
    print("   Container will remain at replicas=0 state.\n")
    
    print("📋 Next steps:")
    print("   1. Verify all 'Required Roles Verification' above are ✅")
    print("   2. Wait about 2-3 minutes")
    print("   3. Run the next cell (Section 7) to start the Container")
    
    print("\n" + "="*60)
    print("✅ Permissions configured successfully!")
    print(f"\n🌐 Endpoint: {FRAMEWORK_ENDPOINT}")
    print(f"\n⏳ Wait for permission propagation before running the next cell!")
else:
    print(f"❌ Deployment failed: {result.stderr}")
    FRAMEWORK_ENDPOINT = None

print("\n" + "="*60)


## 7. Start Agent Framework Service

After permission propagation completes, run this cell to start the Container.

**When to execute:**
- ⏰ **Wait 2-3 minutes** after Section 6 completes
- ⚠️ If permission errors occur: Wait an additional 2-3 minutes and retry


In [ ]:
# Scale to 1 replica
scale_cmd = f"""
az containerapp update \
    --name {framework_app_name} \
    --resource-group {RESOURCE_GROUP} \
    --min-replicas 1 \
    --max-replicas 1
"""

print("🚀 Scaling to 1 replica...")
result = subprocess.run(scale_cmd, shell=True, capture_output=True, text=True, timeout=120)

if result.returncode == 0:
    print("✅ Agent Framework Service started successfully!")
    print(f"\n🌐 Endpoint: {FRAMEWORK_ENDPOINT}")
    print("\n💡 It takes about 30 seconds for the Container to start.")
    print(f"   View logs: az containerapp logs show --name {framework_app_name} --resource-group {RESOURCE_GROUP} --tail 50")
else:
    print(f"❌ Failed to start: {result.stderr}")

print("\n" + "="*60)


## 8. Test Deployed Agent Framework

**✅ Foundry Agent vs Agent Framework - Monitoring & Tracing:**

**Foundry Agent (Lab 3):**
- ✅ Automatic Application Analytics collection
- ✅ Automatic Tracing activation (Azure Agent Service built-in feature)
- ✅ Execution flow viewable in AI Foundry portal

**Agent Framework (Lab 4 - Current):**
- ✅ **Application Analytics collection available** (OpenTelemetry implemented)
- ✅ **Tracing enabled** (custom instrumentation code implemented)
- ✅ **Execution flow viewable in AI Foundry portal**
- 📊 **Additional implemented features:**
  - Router intent classification tracking
  - Executor execution time measurement
  - Tool Agent MCP call detailed tracking
  - Research Agent RAG search tracking
  - Orchestrator parallel execution tracking

> 💡 **Advantage of Agent Framework**:  
> Code-centric approach enables **complete customization**,  
> and direct OpenTelemetry implementation provides **fine-grained monitoring**.

**Tracing hierarchy structure:**
```
📊 api.chat (HTTP Request)
  └─ 🤖 agent_framework.workflow
      ├─ 🧭 workflow.router (Intent Classification)
      │   ├─ router.method: rule_based / ai_based
      │   └─ router.intent: tool / research / orchestrator / general
      │
      └─ ⚙️ workflow.executor.{type}
          ├─ tool_agent.execute → mcp_call
          ├─ research_agent.execute → search + generate
          ├─ general_agent.execute
          └─ orchestrator.parallel_execution
```

**Workflow execution flow:**
1. Router Executor: Intent classification (tool/research/orchestrator/general)
2. Message routing to corresponding Executor
3. Output collection through Workflow Context
4. Return integrated response
5. **All steps recorded in Azure AI Foundry Tracing**


In [ ]:
import requests
import json

print("=== Testing Deployed Agent Framework Service ===\n")

if not FRAMEWORK_ENDPOINT:
    print("❌ FRAMEWORK_ENDPOINT is not set!")
    print("   Please run Section 6 first.\n")
else:
    print(f"🌐 Agent Framework Endpoint: {FRAMEWORK_ENDPOINT}\n")
    
    # 1. Health check
    print("1️⃣ Health Check:")
    try:
        response = requests.get(f"{FRAMEWORK_ENDPOINT}/health", timeout=10)
        if response.status_code == 200:
            print(f"   ✅ Health: {response.json()}\n")
        else:
            print(f"   ❌ Health check failed: {response.status_code}\n")
    except Exception as e:
        print(f"   ❌ Error: {e}\n")
    
    # 2. Check root endpoint
    print("2️⃣ Agent Framework Status:")
    try:
        response = requests.get(f"{FRAMEWORK_ENDPOINT}/", timeout=10)
        if response.status_code == 200:
            status = response.json()
            print(f"   ✅ Status: {status.get('status')}")
            print(f"   📋 Framework: {status.get('framework')}")
            print(f"   🔧 Agents:")
            for agent_name, available in status.get('agents', {}).items():
                icon = "✅" if available else "❌"
                print(f"      {icon} {agent_name}: {available}")
            print()
        else:
            print(f"   ❌ Status check failed: {response.status_code}\n")
    except Exception as e:
        print(f"   ❌ Error: {e}\n")
    
    print("="*70)
    print("\n💡 Agent Framework Service is running normally!")
    print("   Test Workflow execution in the next cell.\n")
    print("   MCP server uses wttr.in API to provide real-time weather information.\n")
    print("="*70)


## 9. Test Workflow Pattern (Various Questions)

Send various questions to the deployed Agent Framework to test Workflow execution.

**Router intent classification:**
- **tool**: Simple tool execution (weather)
- **research**: Simple knowledge search (RAG)
- **orchestrator**: Complex query (tool + knowledge)
- **general**: General conversation

---

### 📚 Research Agent Citation Feature

When Research Agent answers using Azure AI Search, it **automatically displays citations (sources)**:

**Citation format:**
- `【1:0†source】` = First search result document
- `【2:0†source】` = Second search result document
- `【3:0†source】` = Third search result document

**Example response:**
```
📚 [RAG-based Answer]

RAG pattern is a retrieval-augmented generation approach【1:0†source】.
Main advantages include improved accuracy and reduced hallucination【2:0†source】.
It can perform hybrid search by integrating with Azure AI Search【1:0†source】【3:0†source】.
```

**How it works:**
1. Research Agent searches knowledge base with Azure AI Search
2. Top 5 search results injected into LLM context ([Document 1], [Document 2] format)
3. Prompt instructs LLM to cite sources in `【N:0†source】` format
4. LLM automatically cites relevant documents when generating answer
5. Citations are naturally included in the answer text

> 💡 **Difference from Lab 3:** Lab 3's Azure AI Foundry `AzureAISearchTool` automatically generates citations, while Lab 4 implements the same format (`【N:0†source】`) with custom code to provide a consistent user experience.


In [ ]:
# Test Agent Framework Workflow
import requests
import json
import time
import statistics

print("=== Agent Framework Workflow Test ===\n")

# Test cases (3 total)
test_cases = [
    {
        "message": "Hello",
        "description": "General Executor (general conversation) - Warmup"
    },
    {
        "message": "Please tell me the current weather in Seoul. Include temperature, feels-like temperature, weather condition, humidity, and wind information.",
        "description": "Tool Executor (weather query - wttr.in API)"
    },
    {
        "message": "Please recommend tourist attractions in Jeju Island",
        "description": "Research Executor (knowledge search - Jeju Island travel)"
    }
]

success_count = 0
fail_count = 0
latencies = []

for i, test in enumerate(test_cases, 1):
    print(f"{'='*70}")
    print(f"[Test {i}/{len(test_cases)}]")
    print(f"Question: {test['message']}")
    print(f"Expected path: {test['description']}")
    print(f"{'='*70}")
    
    try:
        start_req = time.perf_counter()
        response = requests.post(
            f"{FRAMEWORK_ENDPOINT}/chat",
            json={"message": test['message']},
            headers={"Content-Type": "application/json"},
            timeout=90
        )
        elapsed_req = (time.perf_counter() - start_req) * 1000
        
        if response.status_code == 200:
            result = response.json()
            full_resp = result.get('response', 'No response') or ''
            print(f"\n✅ Response successful (HTTP {response.status_code}) - {elapsed_req:.1f} ms")
            latencies.append(elapsed_req)
            print(f"\n📝 Workflow response:")
            print(full_resp)
            print()
            success_count += 1
        else:
            print(f"\n❌ Request failed (HTTP {response.status_code})")
            print(f"Error: {response.text[:200]}")
            print()
            fail_count += 1
        
    except Exception as e:
        print(f"\n❌ Error occurred: {e}")
        print()
        fail_count += 1
    
    if i < len(test_cases):
        time.sleep(2)

print("="*70)
print(f"\n📊 Test Results:")
print(f"   ✅ Success: {success_count}/{len(test_cases)}")
print(f"   ❌ Failed: {fail_count}/{len(test_cases)}")
print(f"\n💡 Weather information is provided in real-time from wttr.in API.")


## 📍 Next Steps

You've completed deploying the MAF-based Agent! Now proceed to the next notebooks in order:

1. **Notebook 05**: MAF Workflow Patterns Practice (`05_maf_workflow_patterns.ipynb`)
2. **Notebook 06**: MAF Dev UI Practice (`06_maf_dev_ui.ipynb`)
3. **Notebook 07**: Agent Evaluation (`07_evaluate_agents.ipynb`)
